In [1071]:
import requests, zipfile, io
import pandas as pd
import numpy as np
import bokeh.plotting as bpl
import bokeh.palettes as bpt
import bokeh.layouts as bly
import colorcet as cc
from scipy.integrate import odeint
from scipy.integrate import solve_ivp
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_log_error, mean_squared_error

In [2]:
bpl.output_notebook()

Loading BokehJS ...

Definición del modelo de CDMX, los detalles del modelo se encuentran en [https://modelo.covid19.cdmx.gob.mx/modelo-epidemico](https://modelo.covid19.cdmx.gob.mx/modelo-epidemico)

In [3]:
# Susceptibles equation
def dS_dt(S, I, R_t, t_inf):
    return -(R_t / t_inf) * I * S

# Expuestos equation
def dE_dt(S, E, I, R_t, t_inf, t_inc):
    return (R_t / t_inf) * I * S - (E / t_inc)

# Infectados equation
def dI_dt(I, E, t_inc, t_inf):
    return (E / t_inc) - (I / t_inf)

#Leves equation
def dL_dt(I, L, p_grave, t_inf, t_rl):
    return (1-p_grave)*(I /t_inf) - (L / t_rl)

#Graves equation
def dG_dt(I, G, p_grave, t_inf, t_hosp):
    return p_grave*(I / t_inf) - (G / t_hosp)

# Hospializados equation
def dH_dt(G, H, t_hosp, p_icu, t_rh, t_icu):
    return (G / t_hosp) - (1-p_icu)*(H / t_rh) - (p_icu)*(H / t_icu)

# ICU equation
def dICU_dt(H, ICU, p_icu, t_icu, p_m, t_ricu, t_m):
    return p_icu*(H / t_icu) - (1-p_m)*(ICU / t_ricu) - p_m*(ICU / t_m)

# Recovered equation
def dR_dt(L, H, ICU, t_rl, p_icu, t_rh, p_m, t_ricu):
    return (L / t_rl) + (1-p_icu)*(H / t_rh) + (1-p_m)*(ICU / t_ricu)

# Deaths equation
def dD_dt(ICU, t_m, p_m):
    return p_m*(ICU / t_m)


def CDMX_model(t, y, R_t, t_inf=2.9, t_inc=5.2, t_rh=12, t_rl=14, t_hosp=4, t_icu=1, t_ricu=7, t_m=8, p_m=.03, p_grave=.138, p_icu=0.05):

    if callable(R_t):
        reprod = R_t(t)
    else:
        reprod = R_t
        
    S, E, I, L, G, H, ICU, R, D = y
    
    S_out = dS_dt(S, I, reprod, t_inf)
    E_out = dE_dt(S, E, I, reprod, t_inf, t_inc)
    I_out = dI_dt(I, E, t_inc, t_inf)
    L_out = dL_dt(I, L, p_grave, t_inf, t_rl)
    G_out = dG_dt(I, G, p_grave, t_inf, t_hosp)
    H_out = dH_dt(G, H, t_hosp, p_icu, t_rh, t_icu)
    ICU_out = dICU_dt(H, ICU, p_icu, t_icu, p_m, t_ricu, t_m)
    R_out = dR_dt(L, H, ICU, t_rl, p_icu, t_rh, p_m, t_ricu)
    D_out = dD_dt(ICU, t_m, p_m)
    return [S_out, E_out, I_out, L_out, G_out, H_out, ICU_out, R_out, D_out]

Ejecución del modelo de CDMX con los parámetros que reportan, hay dos instancias, una sin intervenciones y la otra con 3 intervenciones

In [659]:
N = 22000000
n_infected = 556*2
max_days = 200
inicio = '2020-03-19'
tiempos = pd.date_range(start=inicio, periods=max_days).values
serie_tiempos = pd.Series(tiempos)

initial_state = [(N - n_infected)/ N, 0, n_infected / N, 0, 0, 0, 0, 0, 0]

R_0 = 3.8
t_inf=2.9
t_inc=5.2
t_rh=12
t_rl=14
t_hosp=4
t_icu=1
t_ricu = 7
t_m=8
p_m=0.65
p_grave=0.138
p_icu=0.05

intervenciones = [{"R":2.83,"fecha":inicio},
                  {"R":2.2,"fecha":"2020-03-22"},
                  {"R":0.95,"fecha":"2020-04-01"},
                  {"R":1.6,"fecha":"2020-04-13"},
                  {"R":0.95,"fecha":"2020-04-20"}]

def rep(t):
    rfinal = R_0
    for intervencion in intervenciones:
        tint = serie_tiempos[(serie_tiempos-pd.Timestamp(intervencion["fecha"])).dt.days==0].index[0]
        if (t > tint):
            rfinal = intervencion["R"]
    return rfinal

args_sin_interv = (R_0, t_inf, t_inc, t_rh, t_rl, t_hosp, t_icu, t_ricu, t_m, p_m, p_grave, p_icu)
args_con_interv = (rep, t_inf, t_inc, t_rh, t_rl, t_hosp, t_icu, t_ricu, t_m, p_m, p_grave, p_icu)

sol_sin_interv = solve_ivp(CDMX_model, [0, max_days], initial_state, args=args_sin_interv, t_eval=np.arange(max_days))
sol_con_interv = solve_ivp(CDMX_model, [0, max_days], initial_state, args=args_con_interv, t_eval=np.arange(max_days))

In [660]:
labels = ["Susceptibles","Expuestos","Infectados","Leves","Graves","Hospitalizados","ICUs","Recuperados","Defunciones"]
colores = {"Susceptibles":"#969696",
           "Recuperados":"Green",
           "Expuestos":"Pink",
           "Infectados":"Red",
           "Leves":"#807DBA",
           "Graves":"#54278F",
           "Hospitalizados":"#CC4C02",
           "ICUs":"#662506",
           "Defunciones":"#525252"}

In [661]:
solucion_sin = pd.DataFrame(sol_sin_interv.y.T*N,columns=labels,index=sol_sin_interv.t)
solucion_con = pd.DataFrame(sol_con_interv.y.T*N,columns=labels,index=sol_con_interv.t)

In [662]:
solucion_sin["Fecha"] = tiempos
solucion_con["Fecha"] = tiempos

In [663]:
solucion_con["R_t"] = pd.Series([rep(ti) for ti in sol_con_interv.t],index=sol_con_interv.t)

In [664]:
solucion_sin[solucion_sin["Fecha"]>"2020-04-01"]

,Susceptibles,Expuestos,Infectados,Leves,Graves,Hospitalizados,ICUs,Recuperados,Defunciones,Fecha
14,2.192687e+07,40777.038529,13371.245243,12802.045827,1310.043894,880.233552,114.879001,3.847460e+03,31.894097,2020-04-02
15,2.190711e+07,51727.032899,16961.131891,16271.449378,1662.582166,1119.383271,147.561806,4.962367e+03,42.471225,2020-04-03
16,2.188214e+07,65493.066317,21564.312070,20651.496378,2105.453167,1424.285630,188.825600,6.381451e+03,56.051051,2020-04-04
17,2.185047e+07,82907.954216,27430.238970,26208.090063,2666.358529,1812.358088,241.146334,8.188692e+03,73.460560,2020-04-05
18,2.181026e+07,105058.351372,34826.005867,33279.820324,3382.153135,2303.657282,307.628523,1.048528e+04,95.700885,2020-04-06
...,...,...,...,...,...,...,...,...,...,...
195,5.395168e+05,2.782056,-16.080566,587.097457,5.974264,-5.901763,1.680581,2.075023e+07,709678.770529,2020-09-30
196,5.395175e+05,1.440237,-8.323521,540.177793,3.092472,-3.019413,1.041372,2.075027e+07,709678.932148,2020-10-01
197,5.395178e+05,0.874827,-5.055191,500.327946,1.878250,-1.810388,0.747436,2.075031e+07,709679.016785,2020-10-02
198,5.395177e+05,0.968861,-5.599332,466.704675,2.080372,-2.022551,0.746986,2.075034e+07,709679.035883,2020-10-03


In [929]:
p_sin = bpl.figure(x_axis_type="datetime",plot_width=800,plot_height=600,title="Modelo CDMX")

In [930]:
for label in labels:
    p_sin.line(x=solucion_sin["Fecha"],y=solucion_sin[label]/1000,color=colores[label],line_width=3,legend_label=label,line_alpha=0.5)

In [931]:
p_sin.xaxis.axis_label = 'Fecha'
p_sin.yaxis.axis_label = 'Miles de personas'
p_sin.legend.location = "center_right"

In [932]:
bpl.show(p_sin)

In [941]:
rp = bpl.figure(x_axis_type="datetime",plot_width=800,plot_height=600,title="Modelo CDMX",y_range=(0,4))

In [942]:
rp.line(x=solucion_con["Fecha"],y=solucion_con["R_t"],color="black",line_width=3,legend_label="R(t)",line_alpha=0.8)

GlyphRenderer(id='191538', ...)

In [943]:
rp.xaxis.axis_label = 'Fecha'
rp.yaxis.axis_label = 'R_t'
rp.legend.location = "center_right"

In [944]:
bpl.show(rp)

In [673]:
p_con = bpl.figure(x_axis_type="datetime",plot_width=800,plot_height=600,title="Modelo CDMX")

In [674]:
for label in ["ICUs"]:
    p_con.line(x=solucion_con["Fecha"],y=solucion_con[label]/1000,color=colores[label],line_width=3,legend_label=label,line_alpha=0.5,line_dash="dotted")

In [675]:
p_con.xaxis.axis_label = 'Fecha'
p_con.yaxis.axis_label = 'Miles de personas'
p_con.legend.location = "center_right"

In [676]:
bpl.show(p_con)

Definición de función para evaluar el modelo en función de R_0 y calculando el error a los datos reales. esto es la función que usaremos para ajustar R_0 a los datos. Se puede seguir una estrategia similar si se desean ajustar más parámetros. Dos parámetros son importantes, los pesos, el primero 't_pesos' indica si el error entre los datos y el modelo es pesado dando prioridad a los datos que estén más cercanos al último día de ajuste. El segundo indica los pesos para cada una de las series a ajustar, las series a ajustar hasta el momento son Confirmados acumulados, Defunciones acumuladas, Casos graves. Se consideran varias series para tratar de quitar dependencia a la tasa de detección de casos.

In [1027]:
def evaluador(R_0, casos, N, dias_prediccion=0,t_pesos=True,pesos_ajuste_CDG=[1,1,1],regresar_solucion=False):
    inicio = casos.index[0]
    n_infected = casos["Confirmados"].fillna(0)[inicio]
    max_days = len(casos) + dias_prediccion
    tiempos = pd.date_range(start=inicio, periods=max_days).values
    serie_tiempos = pd.Series(tiempos)
    
    initial_state = [(N - n_infected)/ N, 0, n_infected / N, 0, 0, 0, 0, 0, 0]

    t_inf=2.9
    t_inc=5.2
    t_rh=12
    t_rl=14
    t_hosp=4
    t_icu=1
    t_ricu = 7
    t_m=8
    p_m=0.65
    p_grave=0.138
    p_icu=0.05
    
    args = (R_0, t_inf, t_inc, t_rh, t_rl, t_hosp, t_icu, t_ricu, t_m, p_m, p_grave, p_icu)
    
    sol = solve_ivp(CDMX_model, [0, max_days], initial_state, args=args, t_eval=np.arange(max_days))
    
    labels = ["Susceptibles","Expuestos","Infectados","Leves","Graves","Hospitalizados","ICUs","Recuperados","Defunciones"]
    
    solucion = pd.DataFrame(sol.y.T*N,columns=labels,index=tiempos)
    solucion["Defunciones"] = np.clip(solucion["Defunciones"],0,np.inf)
    solucion["Graves"] = np.clip(solucion["Graves"],0,np.inf)
    solucion["Confirmados_acumulados"] = np.clip(solucion[["Infectados","Leves","Graves","Hospitalizados","ICUs","Recuperados","Defunciones"]].sum(axis=1),0,np.inf)
    
    if t_pesos:
        pesos = pd.Series(1/((casos.index-casos.index[-1]).days*-1+1),index=casos.index)
    else:
        pesos = pd.Series(1,index=casos.index)
    
    error_confirmados = mean_squared_log_error(casos["Confirmados_acumulados"],solucion["Confirmados_acumulados"][casos.index].fillna(0),pesos)
    error_decesos = mean_squared_log_error(casos["Decesos_acumulados"],solucion["Defunciones"][casos.index].fillna(0),pesos)
    error_graves = mean_squared_log_error(casos["Graves"].fillna(0),solucion["Graves"][casos.index].fillna(0),pesos)
    
    errores = [error_confirmados,error_decesos,error_graves]
        
    error_final = np.dot(pesos_ajuste_CDG,errores)/len(errores)
    
    if regresar_solucion:
        return error_final, solucion
    else:
        return error_final

Obtención de los últimos datos para México

In [1627]:
urlnew = "http://187.191.75.115/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip"
fecha_archivo = pd.Timestamp("2020-05-18")

In [1624]:
archivo = fecha_archivo.strftime("%y%m%d")+"COVID19MEXICO.csv"

In [1625]:
r = requests.get(urlnew)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall(path="./datos_federales/")

In [1628]:
data = pd.read_csv('./datos_federales/' + archivo,encoding="latin1", low_memory=False)

In [1629]:
data["FECHA_ACTUALIZACION"] = pd.to_datetime(data["FECHA_ACTUALIZACION"],format="%Y-%m-%d")
data["FECHA_INGRESO"] = pd.to_datetime(data["FECHA_INGRESO"],format="%Y-%m-%d")
data["FECHA_DEF"] = pd.to_datetime(data["FECHA_DEF"].replace({"9999-99-99":None}),format="%Y-%m-%d")
data["FECHA_SINTOMAS"] = pd.to_datetime(data["FECHA_SINTOMAS"],format="%Y-%m-%d")

In [1630]:
data[:3]

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2020-05-15,04ba92,2,3,15,2,15,15,29,1,...,2,2,2,1,2,1,99,MÃ©xico,99,97
1,2020-05-15,12fb08,2,3,27,1,27,27,4,1,...,2,2,2,2,1,1,99,MÃ©xico,99,97
2,2020-05-15,13889a,2,3,27,1,27,27,12,1,...,2,1,2,2,1,1,99,MÃ©xico,99,97


Filtrado de datos

In [1631]:
sun_data_mun = pd.read_csv("./datos_SUNs/Proy_ZM_2015_mun.csv",encoding="latin1")
sun_data = pd.read_csv("./datos_SUNs/Proy_ZM_2015.csv",encoding="latin1")

In [1632]:
sun_data_mun["CVE_MUN"] = sun_data_mun["CVE_MUN"].astype(str).str.zfill(5)

In [1633]:
suns = pd.merge(sun_data[["CVE_SUN","NOM_SUN","P_2020"]],sun_data_mun.groupby("CVE_SUN")["CVE_MUN"].unique().to_frame("claves").reset_index(),how="outer",on="CVE_SUN")

In [1634]:
suns.sort_values("P_2020",ascending=False)["NOM_SUN"].unique()

array(['Valle de México', 'Guadalajara', 'Monterrey', 'Puebla-Tlaxcala',
       'Toluca', 'Tijuana', 'León', 'Querétaro', 'Juárez', 'La Laguna',
       'Mérida', 'San Luis Potosí', 'Aguascalientes', 'Mexicali',
       'Cuernavaca', 'Saltillo', 'Chihuahua', 'Tampico', 'Morelia',
       'Culiacán', 'Veracruz', 'Hermosillo', 'Acapulco',
       'Tuxtla Gutiérrez', 'Villahermosa', 'Cancún', 'Reynosa', 'Xalapa',
       'Celaya', 'Oaxaca', 'Durango', 'Pachuca', 'Tlaxcala-Apizaco',
       'Poza Rica', 'Matamoros', 'Ensenada', 'Mazatlán', 'Cuautla',
       'Tepic', 'Orizaba', 'Puerto Vallarta', 'Nuevo Laredo',
       'Minatitlán', 'Colima-Villa de Álvarez', 'Zacatecas-Guadalupe',
       'Monclova-Frontera', 'Coatzacoalcos', 'Tapachula',
       'Ciudad Victoria', 'Tehuacán', 'Córdoba', 'Chilpancingo',
       'Campeche', 'La Paz', 'Zamora', 'Tulancingo', 'La Piedad-Pénjamo',
       'Chetumal', 'Nogales', 'Tula', 'Guaymas',
       'San Francisco del Rincón', 'Piedras Negras', 'Delicias',
       'G

In [1635]:
zona_met = 'Valle de México'
z_urbana =suns[suns["NOM_SUN"]==zona_met]
poblacion = z_urbana["P_2020"].values[0]
z_urbana

,CVE_SUN,NOM_SUN,P_2020,claves
18,M09.01,Valle de México,21942666,"[09002, 09003, 09004, 09005, 09006, 09007, 090..."


In [1636]:
data["CVE_MUN_RES"] = data["ENTIDAD_RES"].astype(str).str.zfill(2) + data["MUNICIPIO_RES"].astype(str).str.zfill(3)

In [1637]:
data_zurb = data[(data["RESULTADO"]==1)&(data["CVE_MUN_RES"].isin(z_urbana["claves"].values[0]))]

In [1638]:
#data_zurb = data[(data["RESULTADO"]==1)&(data["ENTIDAD_RES"]==22)]

In [1639]:
data_zurb[:3]

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI,CVE_MUN_RES
0,2020-05-15,04ba92,2,3,15,2,15,15,29,1,...,2,2,1,2,1,99,MÃ©xico,99,97,15029
3,2020-05-15,088d98,2,4,15,1,30,15,20,2,...,2,2,2,99,1,99,MÃ©xico,99,2,15020
5,2020-05-15,09faca,2,4,15,2,15,15,70,1,...,2,2,2,99,1,99,MÃ©xico,99,97,15070


In [1640]:
time_index = pd.date_range(data_zurb["FECHA_SINTOMAS"].min(), fecha_archivo, freq='1D')

In [1641]:
casos = data_zurb.groupby("FECHA_SINTOMAS").size().to_frame("Confirmados").reindex(time_index)

In [1642]:
casos["Graves"] = data_zurb[data_zurb["TIPO_PACIENTE"]==2].groupby(["FECHA_SINTOMAS"]).size()

In [1643]:
casos["Decesos"] = data_zurb.groupby("FECHA_DEF").size()

In [1644]:
casos["Confirmados_acumulados"] = casos["Confirmados"].fillna(0).cumsum()
casos["Decesos_acumulados"] = casos["Decesos"].fillna(0).cumsum()
casos["Graves_acumulados"] = casos["Graves"].fillna(0).cumsum()

In [1645]:
casos[-3:]

,Confirmados,Graves,Decesos,Confirmados_acumulados,Decesos_acumulados,Graves_acumulados
2020-05-16,NaN,NaN,NaN,19223.0,1582.0,7623.0
2020-05-17,NaN,NaN,NaN,19223.0,1582.0,7623.0
2020-05-18,NaN,NaN,NaN,19223.0,1582.0,7623.0


Graficado de datos para Zona Urbana

In [1646]:
inifecha = pd.Timestamp("2020-02-14")
finfecha = pd.Timestamp("2020-05-15")
maxcas = casos["Confirmados_acumulados"][finfecha]
maxcasd = casos["Confirmados"][inifecha:finfecha].max()

In [1647]:
s = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=600,x_range=(inifecha,finfecha),y_range=(0,maxcas))

In [1648]:
s.quad(left=time_index,right=time_index+pd.Timedelta("1 days")*0.9,top=casos["Confirmados_acumulados"],bottom=0)
s.quad(left=time_index,right=time_index+pd.Timedelta("1 days")*0.9,top=casos["Graves_acumulados"],bottom=0,color="green")
s.quad(left=time_index,right=time_index+pd.Timedelta("1 days")*0.9,top=casos["Decesos_acumulados"],bottom=0,color="red")

GlyphRenderer(id='634828', ...)

In [1649]:
bpl.show(s)

In [1653]:
s0 = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=600,x_range=(inifecha,finfecha),y_range=(0,maxcasd))

In [1654]:
s0.quad(left=time_index,right=time_index+pd.Timedelta("1 days")*0.9,top=casos["Confirmados"],bottom=0)
s0.quad(left=time_index,right=time_index+pd.Timedelta("1 days")*0.9,top=casos["Graves"],bottom=0,color="green")
s0.quad(left=time_index,right=time_index+pd.Timedelta("1 days")*0.9,top=casos["Decesos"],bottom=0,color="red")

GlyphRenderer(id='648139', ...)

In [1655]:
bpl.show(s0)

Ejecución del modelo con la R_0 que reportan en CDMX

In [1656]:
fecini = pd.Timestamp("2020-02-17")
fecfin = pd.Timestamp("2020-03-22")
dias_pred = 0
fecpred = fecfin + pd.Timedelta(str(dias_pred) + " days")

In [1657]:
casos_filt = casos.loc[fecini:fecfin,:]
casos_pred = casos.loc[fecfin:fecpred,:]

In [1658]:
casos_filt[:3]

,Confirmados,Graves,Decesos,Confirmados_acumulados,Decesos_acumulados,Graves_acumulados
2020-02-17,1.0,NaN,NaN,2.0,0.0,0.0
2020-02-18,1.0,NaN,NaN,3.0,0.0,0.0
2020-02-19,1.0,NaN,NaN,4.0,0.0,0.0


In [1659]:
R_0a = 2.83
error, solucion = evaluador(R_0a,casos_filt,poblacion,dias_pred,True,[1,1,1],True)

In [1660]:
solucion[-10:]

,Susceptibles,Expuestos,Infectados,Leves,Graves,Hospitalizados,ICUs,Recuperados,Defunciones,Confirmados_acumulados
2020-03-13,2.194249e+07,94.429766,22.453868,40.089829,4.155192,2.476061,0.491450,15.911853,0.200822,85.779076
2020-03-14,2.194245e+07,115.509113,24.829087,48.437636,5.120220,2.857577,0.590958,18.895404,0.240394,100.971276
2020-03-15,2.194243e+07,109.814858,43.892366,50.764954,4.579414,4.038075,0.645964,23.124735,0.294642,127.340152
2020-03-16,2.194240e+07,103.150645,66.472978,53.550848,3.942646,5.437200,0.711408,28.144217,0.359067,158.618364
2020-03-17,2.194234e+07,145.694737,66.588035,69.308553,5.956870,5.902225,0.892593,32.973024,0.423692,182.044992
2020-03-18,2.194225e+07,210.760782,59.015478,91.606376,9.147570,6.108695,1.137493,38.378160,0.496995,205.890768
2020-03-19,2.194217e+07,249.298117,70.479471,108.520032,10.810642,7.269758,1.348664,45.657233,0.593105,244.678905
2020-03-20,2.194211e+07,256.973438,104.220073,119.200347,10.694688,9.549849,1.521139,55.149690,0.716175,301.051961
2020-03-21,2.194203e+07,273.502775,140.351787,133.709692,10.964278,12.080862,1.741085,66.253119,0.860559,365.961383
2020-03-22,2.194189e+07,349.947034,153.172125,164.953083,14.404256,13.737811,2.118716,78.158090,1.018547,427.562628


Graficado de la solución, se puede apreciar que el valor de R_0 no refleja bien el crecimiento que tienen los datos reales

In [1661]:
p = bpl.figure(x_axis_type="datetime",plot_width=800,plot_height=600,title="Modelo CDMX, Zona Metropolitana de " + zona_met,y_axis_type="log")

In [1662]:
p.line(x=casos_filt.index,y=casos_filt["Confirmados_acumulados"],color="#0B9A53",line_width=3,legend_label="Casos a ajustar",line_dash="dashed")
p.line(x=casos_pred.index,y=casos_pred["Confirmados_acumulados"],color="#3999E4",line_width=3,legend_label="Casos a predecir",line_dash="dotted")
p.line(x=solucion.index,y=solucion["Confirmados_acumulados"],color="#9B0E63",line_width=3,legend_label="Casos confirmados, modelo R_0 = " + str(R_0a))
for label in ["Infectados","Leves","Graves","Hospitalizados","ICUs","Recuperados","Defunciones"]:
    p.line(x=solucion.index,y=solucion[label],color=colores[label],line_width=3,legend_label=label)

In [1663]:
p.xaxis.axis_label = 'Fecha'
p.yaxis.axis_label = 'Casos'
p.legend.location = "bottom_right"

In [1664]:
bpl.show(p)

Ajuste del modelo para obtener la mejor R_0

In [1665]:
pesos_ajuste_CDG = [1,1,1]
ajuste = minimize(evaluador,[2.8],args=(casos_filt,poblacion,0,True,pesos_ajuste_CDG,False),method='L-BFGS-B')
ajuste

      fun: 0.13825339540361584
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([-3.33066907e-08])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 10
      nit: 3
   status: 0
  success: True
        x: array([3.0847544])

In [1666]:
R_f = ajuste.x[0]
R_f

3.084754403220855

In [1667]:
error, solucion = evaluador(R_f,casos_filt,poblacion,0,True,pesos_ajuste_CDG,True)

In [1668]:
error

0.13825339540361584

In [1669]:
solucion[-3:]

,Susceptibles,Expuestos,Infectados,Leves,Graves,Hospitalizados,ICUs,Recuperados,Defunciones,Confirmados_acumulados
2020-03-20,2.194168e+07,465.998797,203.774521,197.984123,17.640927,16.344583,2.331220,85.481582,1.021578,524.578534
2020-03-21,2.194145e+07,581.899208,237.225245,243.624063,22.253858,19.383009,2.851917,103.101216,1.234975,629.674284
2020-03-22,2.194118e+07,727.738076,274.810356,300.109620,28.117360,22.937725,3.491532,124.292369,1.491925,755.250887


Graficación del modelo ajustado

In [1670]:
p_casos = bpl.figure(x_axis_type="datetime",plot_width=600,plot_height=400,title="Modelo CDMX, Zona Metropolitana de " + zona_met)
p_def = bpl.figure(x_axis_type="datetime",plot_width=600,plot_height=400)
p_graves = bpl.figure(x_axis_type="datetime",plot_width=600,plot_height=400)

In [1671]:
p_casos.line(x=casos_filt.index,y=casos_filt["Confirmados_acumulados"],color="#0B9A53",line_width=3,legend_label="Casos",line_dash="dashed")
p_def.line(x=casos_filt.index,y=casos_filt["Decesos_acumulados"],color="#0B9A53",line_width=3,legend_label="Defunciones",line_dash="dashed")
p_graves.line(x=casos_filt.index,y=casos_filt["Graves"],color="#0B9A53",line_width=3,legend_label="Casos graves",line_dash="dashed")
for label in ["Confirmados_acumulados"]:
    p_casos.line(x=solucion.index,y=solucion[label],color="#9B0E63",line_width=3,legend_label="modelo R_0 = " + str(R_f))
for label in ["Defunciones"]:
    p_def.line(x=solucion.index,y=solucion[label],color="#9B0E63",line_width=3,legend_label="modelo R_0 = " + str(R_f))
for label in ["Graves"]:
    p_graves.line(x=solucion.index,y=solucion[label],color="#9B0E63",line_width=3,legend_label="modelo R_0 = " + str(R_f))

In [1672]:
p_casos.xaxis.axis_label = 'Fecha'
p_casos.yaxis.axis_label = 'Casos'
p_casos.legend.location = "top_left"
p_def.xaxis.axis_label = 'Fecha'
p_def.yaxis.axis_label = 'Defunciones'
p_def.legend.location = "top_left"
p_graves.xaxis.axis_label = 'Fecha'
p_graves.yaxis.axis_label = 'Casos Graves'
p_graves.legend.location = "top_left"

In [1673]:
bpl.show(bly.column(p_casos,p_def,p_graves))